Good for a practice notebook insteaed of demo. Change it as a todo type of notebook?

## Setup spark and install modules

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, current_timestamp, window, col

In [6]:
spark = SparkSession.builder.appName("LogsAnalysis").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/04/09 14:20:11 WARN Utils: Your hostname, DELEQ0283302041 resolves to a loopback address: 127.0.1.1; using 172.31.227.62 instead (on interface eth0)
24/04/09 14:20:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 14:20:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/09 14:20:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Setup


### Generate data
For out practice purposes, we will generate fake Apache Access logs.
>1. Open terminal and change directory to "Data" folder
>2. In terminal, run command `python log_generator apache_access_schema.yaml`, add flag `-t` to truncate previously created logs
>3. Based on provided .yaml file, new `log.txt.{id}` file should appear every 5s with 50 lines of fake logs.

## Practice

1. Create DataFrame representing the stream of raw log data lines ariving to `/'log/apache_access` directory

In [8]:
access_lines = spark.readStream.text("../Data/log/apache_access")

2. Parse log data to a DataFrame

In [9]:
# Parse out the common log format to a DataFrame
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}:\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = access_lines.select(regexp_extract('value', hostExp, 1).alias('host'),
                        regexp_extract('value', timeExp, 1).alias('timestamp'),
                        regexp_extract('value', generalExp, 1).alias('method'),
                        regexp_extract('value', generalExp, 2).alias('endpoint'),
                        regexp_extract('value', generalExp, 3).alias('protocol'),
                        regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                        regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

3. Add event time column with current time for window aggregation

In [10]:
LogsEventimeDF = logsDF.withColumn("eventTime", current_timestamp())

4. Define windowDuration and slideDuration write windowed

In [11]:
windowDuration = '2 minutes'
slideDuration = '30 seconds'

In [12]:
WindowedEndpointCountsDF = LogsEventimeDF.groupBy(window(col("eventTime"), windowDuration, slideDuration), col("endpoint")).count()

In [13]:
query = WindowedEndpointCountsDF.writeStream.outputMode("complete").format("console") \
      .queryName("counts").start()

24/04/09 14:20:51 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-dd612191-a903-490b-98a5-f45762824421. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 14:20:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-----------+-----+
|              window|   endpoint|count|
+--------------------+-----------+-----+
|{2024-04-09 14:20...|     /users|    6|
|{2024-04-09 14:20...|      /auth|    3|
|{2024-04-09 14:19...|      /auth|    3|
|{2024-04-09 14:20...| /fieldsets|    5|
|{2024-04-09 14:20...|      /auth|    3|
|{2024-04-09 14:20...| /playbooks|    2|
|{2024-04-09 14:19...| /playbooks|    2|
|{2024-04-09 14:19...|      /auth|    3|
|{2024-04-09 14:19...|    /events|    6|
|{2024-04-09 14:19...|     /lists|    9|
|{2024-04-09 14:20...| /customers|    4|
|{2024-04-09 14:20...|    /alerts|    9|
|{2024-04-09 14:19...| /playbooks|    2|
|{2024-04-09 14:20...|     /lists|    9|
|{2024-04-09 14:19...|   /parsers|    2|
|{2024-04-09 14:20...|/collectors|    4|
|{2024-04-09 14:19...|    /events|    6|
|{2024-04-09 14:20...|    /events|    6|
|{2024-04-09 14:20...|   /parsers|    2|
|

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----------+-----+
|              window|   endpoint|count|
+--------------------+-----------+-----+
|{2024-04-09 14:20...|     /users|    9|
|{2024-04-09 14:20...|      /auth|    8|
|{2024-04-09 14:19...|      /auth|    8|
|{2024-04-09 14:20...| /fieldsets|   10|
|{2024-04-09 14:20...|      /auth|    8|
|{2024-04-09 14:20...| /playbooks|    4|
|{2024-04-09 14:19...| /playbooks|    4|
|{2024-04-09 14:19...|      /auth|    8|
|{2024-04-09 14:19...|    /events|   14|
|{2024-04-09 14:19...|     /lists|   15|
|{2024-04-09 14:20...| /customers|    7|
|{2024-04-09 14:20...|    /alerts|   14|
|{2024-04-09 14:19...| /playbooks|    4|
|{2024-04-09 14:20...|     /lists|   15|
|{2024-04-09 14:19...|   /parsers|    9|
|{2024-04-09 14:20...|/collectors|   10|
|{2024-04-09 14:19...|    /events|   14|
|{2024-04-09 14:20...|    /events|   14|
|{2024-04-09 14:20...|   /parsers|    9|
|

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----------+-----+
|              window|   endpoint|count|
+--------------------+-----------+-----+
|{2024-04-09 14:20...|     /users|   10|
|{2024-04-09 14:20...|      /auth|    8|
|{2024-04-09 14:19...|      /auth|    8|
|{2024-04-09 14:20...| /fieldsets|   11|
|{2024-04-09 14:20...|      /auth|    8|
|{2024-04-09 14:20...| /playbooks|    4|
|{2024-04-09 14:19...| /playbooks|    4|
|{2024-04-09 14:19...|      /auth|    8|
|{2024-04-09 14:19...|    /events|   14|
|{2024-04-09 14:19...|     /lists|   16|
|{2024-04-09 14:20...| /customers|    8|
|{2024-04-09 14:20...|    /alerts|   14|
|{2024-04-09 14:19...| /playbooks|    4|
|{2024-04-09 14:20...|     /lists|   16|
|{2024-04-09 14:19...|   /parsers|    9|
|{2024-04-09 14:20...|/collectors|   10|
|{2024-04-09 14:19...|    /events|   14|
|{2024-04-09 14:20...|    /events|   14|
|{2024-04-09 14:20...|   /parsers|    9|
|

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----------+-----+
|              window|   endpoint|count|
+--------------------+-----------+-----+
|{2024-04-09 14:20...|     /users|   10|
|{2024-04-09 14:20...|      /auth|    8|
|{2024-04-09 14:19...|      /auth|    8|
|{2024-04-09 14:20...| /fieldsets|   11|
|{2024-04-09 14:20...|      /auth|    8|
|{2024-04-09 14:20...| /playbooks|    4|
|{2024-04-09 14:19...| /playbooks|    4|
|{2024-04-09 14:19...|      /auth|    8|
|{2024-04-09 14:19...|    /events|   14|
|{2024-04-09 14:21...|    /events|    3|
|{2024-04-09 14:19...|     /lists|   16|
|{2024-04-09 14:20...| /customers|    8|
|{2024-04-09 14:20...|    /alerts|   14|
|{2024-04-09 14:19...| /playbooks|    4|
|{2024-04-09 14:20...|     /lists|   16|
|{2024-04-09 14:19...|   /parsers|    9|
|{2024-04-09 14:20...|/collectors|   10|
|{2024-04-09 14:19...|    /events|   17|
|{2024-04-09 14:20...|    /events|   17|
|

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+----------+-----+
|              window|  endpoint|count|
+--------------------+----------+-----+
|{2024-04-09 14:20...|    /users|   15|
|{2024-04-09 14:20...|     /auth|   15|
|{2024-04-09 14:19...|     /auth|    8|
|{2024-04-09 14:20...|/fieldsets|   12|
|{2024-04-09 14:20...|     /auth|   15|
|{2024-04-09 14:20...|/playbooks|    6|
|{2024-04-09 14:19...|/playbooks|    6|
|{2024-04-09 14:19...|     /auth|   15|
|{2024-04-09 14:21...|  /parsers|    5|
|{2024-04-09 14:19...|   /events|   14|
|{2024-04-09 14:21...|   /events|   11|
|{2024-04-09 14:19...|    /lists|   20|
|{2024-04-09 14:20...|/customers|   15|
|{2024-04-09 14:20...|   /alerts|   17|
|{2024-04-09 14:21...|    /users|    5|
|{2024-04-09 14:19...|/playbooks|    4|
|{2024-04-09 14:20...|    /lists|   20|
|{2024-04-09 14:21...|/fieldsets|    1|
|{2024-04-09 14:19...|  /parsers|   14|
|{2024-04-09 14:21...| 

5. Classroom cleanup
IMPORTANT: Kill running log_generator script in terminal

In [11]:
query.stop()
spark.stop()